In [9]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [10]:
df = pd.read_csv("../../data/flags/flags.csv", header=None)

flags = set([(df[1][i],df[2][i],df[3][i]) for i in range(0,14)])

colors = ["red","lightblue","blue","green","orange","yellow","white"]

x = []
y = []

for c1 in colors:
    x.append(c1)
    if c1 in [x[0] for x in flags]:
        y.append(1)
    else:
        y.append(0)
    
    for c2 in colors:
        x.append(c1+","+c2)
        if c2 in [x[1] for x in flags if x[0]==c1]:
            y.append(1)
        else:
            y.append(0)
        
        for c3 in colors:
            x.append(c1+","+c2+","+c3)
            if c3 in [x[2] for x in flags if x[0]==c1 and x[1]==c2]:
                y.append(1)
            else:
                y.append(0)

x = np.array(x)

y = np.array(y)

In [11]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=20, split=",")

# updates internal vocabulary based on the lyrics
tokenizer.fit_on_texts(x)

# transform each text in x to a sequence of tokens
x = tokenizer.texts_to_sequences(x)

x = np.array(pad_sequences(x, maxlen = 3))

print(x)

[[0 0 1]
 [0 1 1]
 [1 1 1]
 ...
 [7 7 5]
 [7 7 6]
 [7 7 7]]


In [12]:
num = len(y)

idx = np.random.permutation(num)

x = x[idx]
y = y[idx]

x = x.reshape((x.shape[0], x.shape[1], 1))

n_classes = len(np.unique(y))

np.unique(y)

print(n_classes)

2


In [13]:

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [14]:

def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    return keras.Model(inputs, outputs)


In [15]:
input_shape = x.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0,
    dropout=0,
)

model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-2),
    metrics=["accuracy"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    x,
    y,
    validation_data=(x,y),
    #validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
)

model.evaluate(x, y, verbose=1)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 3, 1)]       0           []                               
                                                                                                  
 layer_normalization_8 (LayerNo  (None, 3, 1)        2           ['input_2[0][0]']                
 rmalization)                                                                                     
                                                                                                  
 multi_head_attention_4 (MultiH  (None, 3, 1)        7169        ['layer_normalization_8[0][0]',  
 eadAttention)                                                    'layer_normalization_8[0][0]']  
                                                                                            

[0.13166435062885284, 0.9548872113227844]

In [16]:
model.save('../models/flags_validator_model')

INFO:tensorflow:Assets written to: ../models/flags_validator_model/assets


INFO:tensorflow:Assets written to: ../models/flags_validator_model/assets
